In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_log_error

import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('D:/IMS Proschool/Business Analytics/Practice/Self/E-Commerce_Participants_Data/Train.csv')
test = pd.read_csv('D:/IMS Proschool/Business Analytics/Practice/Self/E-Commerce_Participants_Data/Test.csv')

In [3]:
train.shape

(2452, 8)

In [4]:
test.shape

(1051, 7)

In [5]:
data = pd.concat([train,test], axis=0, ignore_index=True)
data.shape

(3503, 8)

In [6]:
data.head()

,Product,Product_Brand,Item_Category,Subcategory_1,Subcategory_2,Item_Rating,Date,Selling_Price
0,P-2610,B-659,bags wallets belts,bags,hand bags,4.3,2/3/2017,291.0
1,P-2453,B-3078,clothing,women s clothing,western wear,3.1,7/1/2015,897.0
2,P-6802,B-1810,home decor festive needs,showpieces,ethnic,3.5,1/12/2019,792.0
3,P-4452,B-3078,beauty and personal care,eye care,h2o plus eye care,4.0,12/12/2014,837.0
4,P-8454,B-3078,clothing,men s clothing,t shirts,4.3,12/12/2013,470.0


In [7]:
data.tail()

,Product,Product_Brand,Item_Category,Subcategory_1,Subcategory_2,Item_Rating,Date,Selling_Price
3498,P-9758,B-2543,sports fitness,outdoor adventure,camping hiking,2.0,28/2/2014,NaN
3499,P-11898,B-3197,jewellery,necklaces chains,necklaces,4.7,1/12/2019,NaN
3500,P-904,B-133,automotive,accessories spare parts,car interior exterior,3.5,1/12/2014,NaN
3501,P-1714,B-358,kitchen dining,lighting,bulbs,2.4,23/6/2013,NaN
3502,P-620,B-133,automotive,accessories spare parts,car interior exterior,3.1,1/12/2012,NaN


In [14]:
data_features = data.drop(['Product','Date'], axis=1)

In [15]:
ct = ColumnTransformer([('encoder', OneHotEncoder(), [0,1,2,3,4])], remainder='passthrough') 
data_features = ct.fit_transform(data_features).toarray()
data_features

array([[  0.,   0.,   0., ...,   0.,   0., 291.],
       [  0.,   0.,   0., ...,   0.,   0., 897.],
       [  0.,   0.,   0., ...,   0.,   0., 792.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,  nan],
       [  0.,   0.,   0., ...,   0.,   0.,  nan],
       [  0.,   0.,   0., ...,   0.,   0.,  nan]])

In [19]:
data_encoded = pd.DataFrame(data_features)
data_encoded = data_encoded.rename(columns={1986:"Selling_Price"})
data_encoded.head()

,0,1,2,3,4,5,6,7,8,9,...,1977,1978,1979,1980,1981,1982,1983,1984,1985,Selling_Price
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,291.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,897.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,792.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,837.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,470.0


In [20]:
data_encoded.tail()

,0,1,2,3,4,5,6,7,8,9,...,1977,1978,1979,1980,1981,1982,1983,1984,1985,Selling_Price
3498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3499,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN
3500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3501,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3502,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [21]:
train_features = data_encoded[data_encoded['Selling_Price'].isna() == False]
test_features = data_encoded[data_encoded['Selling_Price'].isna() == True]
train_label = train_features['Selling_Price']

In [22]:
train_features = train_features.drop(['Selling_Price'], axis=1)
test_features = test_features.drop(['Selling_Price'], axis=1)

In [23]:
rf = RandomForestRegressor()
etr = ExtraTreesRegressor()
abr = AdaBoostRegressor()
xgb = XGBRegressor()
knr = KNeighborsRegressor()
cbr = CatBoostRegressor()

In [24]:
rf_score = cross_val_score(rf, train_features, train_label, cv=5, scoring='neg_mean_squared_log_error')

In [25]:
rf_score.mean()

-0.4870928452504503

In [26]:
etr_score = cross_val_score(etr, train_features, train_label, cv=5, scoring='neg_mean_squared_log_error')

In [27]:
etr_score.mean()

-0.5757471782902233

In [28]:
abr_score = cross_val_score(abr, train_features, train_label, cv=5, scoring='neg_mean_squared_log_error')

In [29]:
abr_score.mean()

-1.911848335348023

In [30]:
xgb_score = cross_val_score(xgb, train_features, train_label, cv=5, scoring='neg_mean_squared_log_error')

[20:12:14] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:12:33] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:12:51] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:13:09] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:13:28] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [31]:
xgb_score.mean()

-0.8424749133157686

In [32]:
knr_score = cross_val_score(knr, train_features, train_label, cv=5, scoring='neg_mean_squared_log_error')

In [33]:
knr_score.mean()

-0.771256089222414

In [34]:
cbr_score = cross_val_score(cbr, train_features, train_label, cv=5, scoring='neg_mean_squared_log_error')

Learning rate set to 0.044001
0:	learn: 6555.0642477	total: 201ms	remaining: 3m 20s
1:	learn: 6351.6207456	total: 209ms	remaining: 1m 44s
2:	learn: 6161.0705669	total: 217ms	remaining: 1m 12s
3:	learn: 5985.4449089	total: 225ms	remaining: 56.1s
4:	learn: 5810.4403974	total: 233ms	remaining: 46.4s
5:	learn: 5643.3353810	total: 241ms	remaining: 40s
6:	learn: 5488.6587799	total: 249ms	remaining: 35.4s
7:	learn: 5337.7672417	total: 258ms	remaining: 32.1s
8:	learn: 5196.7184963	total: 262ms	remaining: 28.8s
9:	learn: 5061.4840844	total: 270ms	remaining: 26.8s
10:	learn: 4923.9151039	total: 278ms	remaining: 25s
11:	learn: 4802.7052326	total: 286ms	remaining: 23.5s
12:	learn: 4689.1692801	total: 294ms	remaining: 22.3s
13:	learn: 4583.0743689	total: 301ms	remaining: 21.2s
14:	learn: 4482.1050835	total: 309ms	remaining: 20.3s
15:	learn: 4378.3700357	total: 317ms	remaining: 19.5s
16:	learn: 4287.0203759	total: 325ms	remaining: 18.8s
17:	learn: 4199.1829298	total: 333ms	remaining: 18.2s
18:	learn

157:	learn: 2283.1800248	total: 1.49s	remaining: 7.94s
158:	learn: 2280.5896584	total: 1.5s	remaining: 7.92s
159:	learn: 2273.5873562	total: 1.51s	remaining: 7.92s
160:	learn: 2270.4659335	total: 1.52s	remaining: 7.91s
161:	learn: 2267.7555181	total: 1.53s	remaining: 7.9s
162:	learn: 2264.6420603	total: 1.53s	remaining: 7.88s
163:	learn: 2262.0531405	total: 1.54s	remaining: 7.86s
164:	learn: 2259.2514837	total: 1.55s	remaining: 7.84s
165:	learn: 2256.4561846	total: 1.56s	remaining: 7.83s
166:	learn: 2253.2308573	total: 1.56s	remaining: 7.81s
167:	learn: 2250.5942446	total: 1.57s	remaining: 7.79s
168:	learn: 2247.7824163	total: 1.58s	remaining: 7.77s
169:	learn: 2244.5583424	total: 1.59s	remaining: 7.75s
170:	learn: 2241.9950089	total: 1.6s	remaining: 7.74s
171:	learn: 2240.0927363	total: 1.6s	remaining: 7.72s
172:	learn: 2233.8332848	total: 1.61s	remaining: 7.7s
173:	learn: 2223.7281297	total: 1.62s	remaining: 7.69s
174:	learn: 2219.5309869	total: 1.63s	remaining: 7.68s
175:	learn: 221

318:	learn: 1868.5728396	total: 2.78s	remaining: 5.93s
319:	learn: 1863.8002892	total: 2.79s	remaining: 5.92s
320:	learn: 1861.8107987	total: 2.8s	remaining: 5.92s
321:	learn: 1860.6472618	total: 2.8s	remaining: 5.91s
322:	learn: 1859.4979726	total: 2.81s	remaining: 5.89s
323:	learn: 1858.3522676	total: 2.82s	remaining: 5.88s
324:	learn: 1857.2198451	total: 2.83s	remaining: 5.87s
325:	learn: 1856.0942012	total: 2.84s	remaining: 5.86s
326:	learn: 1854.2339083	total: 2.84s	remaining: 5.85s
327:	learn: 1850.9923960	total: 2.85s	remaining: 5.84s
328:	learn: 1849.8706859	total: 2.86s	remaining: 5.83s
329:	learn: 1848.7599699	total: 2.87s	remaining: 5.82s
330:	learn: 1847.6516660	total: 2.87s	remaining: 5.81s
331:	learn: 1843.0824724	total: 2.88s	remaining: 5.8s
332:	learn: 1841.9809007	total: 2.89s	remaining: 5.79s
333:	learn: 1840.2570934	total: 2.9s	remaining: 5.78s
334:	learn: 1839.1645406	total: 2.9s	remaining: 5.77s
335:	learn: 1838.0734274	total: 2.91s	remaining: 5.76s
336:	learn: 183

479:	learn: 1662.0816760	total: 4.05s	remaining: 4.39s
480:	learn: 1661.4353940	total: 4.06s	remaining: 4.38s
481:	learn: 1660.7898171	total: 4.07s	remaining: 4.37s
482:	learn: 1660.1513291	total: 4.08s	remaining: 4.37s
483:	learn: 1659.5111331	total: 4.09s	remaining: 4.36s
484:	learn: 1658.2811524	total: 4.09s	remaining: 4.35s
485:	learn: 1657.6483395	total: 4.1s	remaining: 4.34s
486:	learn: 1654.6791974	total: 4.11s	remaining: 4.33s
487:	learn: 1653.6421759	total: 4.12s	remaining: 4.32s
488:	learn: 1652.5542944	total: 4.12s	remaining: 4.31s
489:	learn: 1651.9230533	total: 4.13s	remaining: 4.3s
490:	learn: 1648.1220930	total: 4.14s	remaining: 4.29s
491:	learn: 1647.4893987	total: 4.15s	remaining: 4.28s
492:	learn: 1646.8604693	total: 4.16s	remaining: 4.27s
493:	learn: 1646.2347525	total: 4.16s	remaining: 4.26s
494:	learn: 1645.2384554	total: 4.17s	remaining: 4.25s
495:	learn: 1643.0490582	total: 4.18s	remaining: 4.25s
496:	learn: 1642.4264148	total: 4.19s	remaining: 4.24s
497:	learn: 

632:	learn: 1518.0083460	total: 5.3s	remaining: 3.07s
633:	learn: 1517.5772209	total: 5.31s	remaining: 3.06s
634:	learn: 1517.1473309	total: 5.32s	remaining: 3.06s
635:	learn: 1516.7181226	total: 5.32s	remaining: 3.05s
636:	learn: 1516.0383604	total: 5.33s	remaining: 3.04s
637:	learn: 1515.6123765	total: 5.34s	remaining: 3.03s
638:	learn: 1515.1889680	total: 5.35s	remaining: 3.02s
639:	learn: 1514.7666633	total: 5.36s	remaining: 3.01s
640:	learn: 1512.3167211	total: 5.36s	remaining: 3s
641:	learn: 1511.8973643	total: 5.37s	remaining: 3s
642:	learn: 1511.4769886	total: 5.38s	remaining: 2.99s
643:	learn: 1511.0607099	total: 5.39s	remaining: 2.98s
644:	learn: 1510.3712727	total: 5.39s	remaining: 2.97s
645:	learn: 1509.9560301	total: 5.4s	remaining: 2.96s
646:	learn: 1509.1049114	total: 5.41s	remaining: 2.95s
647:	learn: 1508.4504053	total: 5.42s	remaining: 2.94s
648:	learn: 1508.0361095	total: 5.42s	remaining: 2.93s
649:	learn: 1507.6228673	total: 5.43s	remaining: 2.92s
650:	learn: 1507.2

...

330:	learn: 1963.8472807	total: 2.74s	remaining: 5.54s
331:	learn: 1961.3693351	total: 2.75s	remaining: 5.53s
332:	learn: 1959.8721359	total: 2.76s	remaining: 5.52s
333:	learn: 1958.3836712	total: 2.77s	remaining: 5.52s
334:	learn: 1956.9117344	total: 2.77s	remaining: 5.51s
335:	learn: 1954.4852316	total: 2.78s	remaining: 5.5s
336:	learn: 1953.0215027	total: 2.79s	remaining: 5.49s
337:	learn: 1949.3171578	total: 2.8s	remaining: 5.48s
338:	learn: 1947.8644902	total: 2.81s	remaining: 5.47s
339:	learn: 1946.4257391	total: 2.81s	remaining: 5.46s
340:	learn: 1945.0000172	total: 2.82s	remaining: 5.45s
341:	learn: 1940.7437464	total: 2.83s	remaining: 5.44s
342:	learn: 1937.9882843	total: 2.83s	remaining: 5.43s
343:	learn: 1936.5714985	total: 2.84s	remaining: 5.42s
344:	learn: 1933.7279194	total: 2.85s	remaining: 5.41s
345:	learn: 1931.4046844	total: 2.86s	remaining: 5.4s
346:	learn: 1929.9942416	total: 2.87s	remaining: 5.39s
347:	learn: 1924.2944606	total: 2.87s	remaining: 5.38s
348:	learn: 1

491:	learn: 1710.0870351	total: 4.01s	remaining: 4.14s
492:	learn: 1708.8762137	total: 4.01s	remaining: 4.13s
493:	learn: 1708.0771720	total: 4.02s	remaining: 4.12s
494:	learn: 1706.9039704	total: 4.03s	remaining: 4.11s
495:	learn: 1706.1084944	total: 4.04s	remaining: 4.1s
496:	learn: 1705.3206678	total: 4.05s	remaining: 4.09s
497:	learn: 1703.6788321	total: 4.05s	remaining: 4.09s
498:	learn: 1702.8955467	total: 4.06s	remaining: 4.08s
499:	learn: 1701.6363791	total: 4.07s	remaining: 4.07s
500:	learn: 1699.3479467	total: 4.08s	remaining: 4.06s
501:	learn: 1698.5658434	total: 4.08s	remaining: 4.05s
502:	learn: 1697.7902848	total: 4.09s	remaining: 4.04s
503:	learn: 1697.0206831	total: 4.1s	remaining: 4.04s
504:	learn: 1693.9321931	total: 4.11s	remaining: 4.03s
505:	learn: 1692.7078222	total: 4.12s	remaining: 4.02s
506:	learn: 1691.9401136	total: 4.12s	remaining: 4.01s
507:	learn: 1691.1782742	total: 4.13s	remaining: 4s
508:	learn: 1690.2743538	total: 4.14s	remaining: 3.99s
509:	learn: 168

653:	learn: 1555.5929308	total: 5.28s	remaining: 2.79s
654:	learn: 1555.1077760	total: 5.29s	remaining: 2.78s
655:	learn: 1554.6231413	total: 5.29s	remaining: 2.78s
656:	learn: 1554.1427076	total: 5.3s	remaining: 2.77s
657:	learn: 1552.5679518	total: 5.31s	remaining: 2.76s
658:	learn: 1552.0883750	total: 5.32s	remaining: 2.75s
659:	learn: 1551.5356998	total: 5.33s	remaining: 2.74s
660:	learn: 1550.7689721	total: 5.33s	remaining: 2.74s
661:	learn: 1550.2929231	total: 5.34s	remaining: 2.73s
662:	learn: 1548.6585563	total: 5.35s	remaining: 2.72s
663:	learn: 1548.1847595	total: 5.36s	remaining: 2.71s
664:	learn: 1547.7128666	total: 5.37s	remaining: 2.7s
665:	learn: 1547.1698781	total: 5.37s	remaining: 2.69s
666:	learn: 1546.6314036	total: 5.38s	remaining: 2.69s
667:	learn: 1546.1268559	total: 5.39s	remaining: 2.68s
668:	learn: 1545.6599952	total: 5.4s	remaining: 2.67s
669:	learn: 1544.2466858	total: 5.4s	remaining: 2.66s
670:	learn: 1542.5542486	total: 5.41s	remaining: 2.65s
671:	learn: 15

817:	learn: 1445.0323558	total: 6.57s	remaining: 1.46s
818:	learn: 1444.6879029	total: 6.57s	remaining: 1.45s
819:	learn: 1444.3444896	total: 6.58s	remaining: 1.44s
820:	learn: 1444.0005248	total: 6.59s	remaining: 1.44s
821:	learn: 1442.4368286	total: 6.6s	remaining: 1.43s
822:	learn: 1442.0935705	total: 6.61s	remaining: 1.42s
823:	learn: 1441.1677971	total: 6.61s	remaining: 1.41s
824:	learn: 1440.8281870	total: 6.62s	remaining: 1.4s
825:	learn: 1440.4351746	total: 6.63s	remaining: 1.4s
826:	learn: 1440.0668703	total: 6.63s	remaining: 1.39s
827:	learn: 1439.5292231	total: 6.64s	remaining: 1.38s
828:	learn: 1438.4455382	total: 6.65s	remaining: 1.37s
829:	learn: 1438.1066924	total: 6.66s	remaining: 1.36s
830:	learn: 1437.0632889	total: 6.67s	remaining: 1.35s
831:	learn: 1436.7246751	total: 6.67s	remaining: 1.35s
832:	learn: 1436.3872307	total: 6.68s	remaining: 1.34s
833:	learn: 1436.0010264	total: 6.69s	remaining: 1.33s
834:	learn: 1435.6051061	total: 6.7s	remaining: 1.32s
835:	learn: 14

977:	learn: 1358.9625808	total: 7.82s	remaining: 176ms
978:	learn: 1358.6977541	total: 7.83s	remaining: 168ms
979:	learn: 1358.3139471	total: 7.83s	remaining: 160ms
980:	learn: 1358.0510050	total: 7.84s	remaining: 152ms
981:	learn: 1357.7894848	total: 7.85s	remaining: 144ms
982:	learn: 1356.7462865	total: 7.86s	remaining: 136ms
983:	learn: 1356.4858097	total: 7.87s	remaining: 128ms
984:	learn: 1356.1746479	total: 7.87s	remaining: 120ms
985:	learn: 1355.9150114	total: 7.88s	remaining: 112ms
986:	learn: 1355.6548857	total: 7.89s	remaining: 104ms
987:	learn: 1354.5611636	total: 7.9s	remaining: 95.9ms
988:	learn: 1354.3013762	total: 7.91s	remaining: 87.9ms
989:	learn: 1353.6537201	total: 7.91s	remaining: 79.9ms
990:	learn: 1353.1097376	total: 7.92s	remaining: 71.9ms
991:	learn: 1352.8505499	total: 7.93s	remaining: 63.9ms
992:	learn: 1352.4673320	total: 7.93s	remaining: 55.9ms
993:	learn: 1352.2093604	total: 7.94s	remaining: 47.9ms
994:	learn: 1351.3793405	total: 7.95s	remaining: 40ms
995:	

In [35]:
cbr_score.mean()

-0.6678896914883062

In [36]:
X_train, X_val, y_train, y_val = train_test_split(train_features, train_label, train_size=0.8, random_state=1)

In [37]:
rf

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [47]:
params = {'bootstrap': [True,False],
          'criterion': ['mse','mae'],
          'max_depth': [None,2,4,6,8],
          'max_features': [4,6,8,10],
          'min_samples_split': [2,4,5,6],
          'min_samples_leaf': [1,2,4,6],
          'max_leaf_nodes': [None,2,4,6],
          'n_estimators': [10,30,100]}

In [38]:
params2 = {'bootstrap': [False],
          'criterion': ['mae'],
          'max_features': [9,10,11,12],
          'min_samples_split': [4,5,6,7,8]
          }

In [70]:
grid_search = GridSearchCV(rf, param_grid=params, cv=3, scoring='neg_mean_squared_log_error', verbose=1, n_jobs=-1)

In [49]:
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 15360 candidates, totalling 46080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   18.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   28.9s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   47.0s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 4992 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done 6042 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 7192 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 8442 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done 9792 tasks      | elapsed: 13.0min
[Parallel(n_jobs=-1)]: Done 11242 tasks      |

GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, rand...
             iid='deprecated', n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         '

In [39]:
grid_search2 = GridSearchCV(rf, param_grid=params2, cv=5, scoring='neg_mean_squared_log_error', verbose=1, n_jobs=-1)

In [40]:
grid_search2.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 28.0min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 70.2min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [52]:
y_pred = grid_search.predict(X_val)

In [41]:
y_pred2 = grid_search2.predict(X_val)

In [53]:
mean_squared_log_error(y_pred,y_val)

0.6551927895185529

In [42]:
mean_squared_log_error(y_pred2,y_val)

0.6027662252865478

In [54]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                      max_depth=None, max_features=10, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=6, min_weight_fraction_leaf=0.0,
                      n_estimators=10, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [43]:
grid_search2.best_estimator_

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                      max_depth=None, max_features=9, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=8, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [44]:
pars = { "bootstrap":False, "criterion":"mae", "max_features":9, "min_samples_split":8}
rf = RandomForestRegressor(**pars)

In [45]:
rf.fit(X_train,y_train)

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                      max_depth=None, max_features=9, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=8, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [46]:
y_pred3 = rf.predict(X_val)
mean_squared_log_error(y_pred3,y_val)

0.549049018096399

In [48]:
submit = pd.DataFrame(y_pred3, columns=['Selling_Price'])
submit.head(10)

,Selling_Price
0,741.585
1,670.580
2,799.815
3,350.130
4,2832.670
5,1056.950
6,345.860
7,1722.930
8,327.800
9,553.110


In [50]:
submit.to_excel('D:/IMS Proschool/Business Analytics/Practice/Self/E-Commerce_Participants_Data/Final_Submit.xlsx', index=False)